## Life Expectancy Prediction

## Data Preprocessing Stage

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Importing Dataset and rearranging dataset so as to seperate dependent and independent varibles

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
df_data_2 = df_data_2[df_data_2.columns[[3,0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,10,21]]] #rearranging the dataset
X = df_data_2.iloc[:, 1:].values
y = df_data_2.iloc[:, 0].values

Handling Missing values 

In [ ]:
from sklearn.preprocessing import Imputer
Imputer_x = Imputer(missing_values = "NaN", strategy = 'mean', axis = 0)
     #fit imputer mean to missing values that require mean
Imputer_x = Imputer_x.fit(X[:,3:])
X[:,3:] = Imputer_x.transform(X[:,3:])

Imputer_y = Imputer(missing_values = "NaN", strategy = 'mean', axis = 0)
     #fit imputer mean to missing values that require mean
Imputer_y = Imputer_y.fit(y.reshape(-1, 1))
y = Imputer_y.transform(y.reshape(-1, 1))

In [ ]:
print(X)
print(y)

Handling categorical variables

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_country = LabelEncoder()
X[:, 0] = labelencoder_X_country.fit_transform(X[:, 0])

labelencoder_X_year = LabelEncoder()
X[:, 1] = labelencoder_X_year.fit_transform(X[:,1])

labelencoder_X_Status = LabelEncoder()
X[:,2] = labelencoder_X_Status.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [0,1,2])
X= onehotencoder.fit_transform(X).toarray()

## Splitting Dataset

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test,y_train , y_test  = train_test_split(X , y , test_size = 0.25)

## Feature Scaling of values to Scale them

In [ ]:
from sklearn.preprocessing import StandardScaler
scale_x = StandardScaler()
x_train = scale_x.fit_transform(x_train)
x_test = scale_x.transform(x_test)

scale_y = StandardScaler()
y_train = scale_y.fit_transform(y_train.reshape(-1, 1))
y_test = scale_y.transform(y_test.reshape(-1, 1))

## Training ML Model

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(x_train, y_train.ravel())

## Predicting the test set results

In [ ]:
y_pred = regressor.predict(x_test)
np.set_printoptions(precision=2)
print(np.concatenate((scale_y.inverse_transform(y_pred).reshape(len(y_pred),1), scale_y.inverse_transform(y_test).reshape(len(y_test),1)),1))

## Evaluation of model Performance

In [ ]:
from sklearn.metrics import r2_score
r2_score(scale_y.inverse_transform(y_test),scale_y.inverse_transform( y_pred))

## Deployment code

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials={
  "apikey": "FzWa9Ig_0hxvvOy5a_sTnEMf63t4D38MhXkx5XriUjPP",
  "iam_apikey_description": "Auto-generated for key 8bfe8078-d209-4991-9f35-54e855004667",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/2b8912695c064f3caad60ffb9f881945::serviceid:ServiceId-546e0dd9-ad63-4c58-87a5-cf852c1bd2fb",
  "instance_id": "39446403-9d82-46a4-8b1f-e9d53a441f01",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [ ]:
metadata={
    client.repository.ModelMetaNames.DESCRIPTION:'Life Expectancy',
    client.repository.ModelMetaNames.AUTHOR_NAME:'Aaditya',
    client.repository.ModelMetaNames.NAME:"LifeExpSVM",
    client.repository.ModelMetaNames.FRAMEWORK_NAME:"scikit-learn",
    client.repository.ModelMetaNames.FRAMEWORK_VERSION:"0.22"
    
}

In [ ]:
model_details=client.repository.store_model( regressor ,meta_props=metadata)

In [ ]:
published_model_uid=client.repository.get_model_uid(model_details)
published_model_uid

In [ ]:
# Creating deployment
created_deployment=client.deployments.create(published_model_uid,name="SVM")

In [ ]:
# Gettingn Endpoints
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
scoring_endpoint

In [ ]:
client.deployments.list()